In [1]:
import os
import pandas as pd
from datetime import datetime 

In [2]:
# Set the working directory
current_dir = os.getcwd()
print('current_dir: ',current_dir)
parent_dir = os.path.dirname(current_dir)

os.chdir(parent_dir)

print('Current working directory:', os.getcwd())

current_dir:  c:\Users\Utente\Documents\GitHub\SE4GEO-Lab\db_helper
Current working directory: c:\Users\Utente\Documents\GitHub\SE4GEO-Lab


In [ ]:
fileName = "se4g_dashboard_dataset.csv"
folder_out = 'data_prova'
full_path = os.path.join(folder_out, fileName)

country = {'AD': 'Andorra', 
        'SE':'Sweden', 
        'DE':'Germany', 
        'CY':'Undefined', 
        'BE':'Belgium', 
        'FI':'Finland', 
        'ES':'Spain', 
        'CZ':'Czech Republic', 
        'BG':'Bulgaria', 
        'BA':'Bosnia and Herzegovina', 
        'EE':'Estonia', 
        'CH':'Switzerland',
        'AT':'Austria', 
        'DK':'Denmark'}

if os.path.isfile(full_path):
    dashboard_df = pd.read_csv(full_path)
    # Convert 'value_datetime_end' to datetime objects and extract the day
    datetime_objects = dashboard_df['value_datetime_end'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S%z'))

    dashboard_df['month_day'] = datetime_objects.apply(lambda x: x.strftime('%m%d')).astype('int64')
    
    # Compute daily mean of 'value_numeric' for each 'pollutant' and 'network_countrycode'
    daily_mean = dashboard_df.groupby(['pollutant', 'network_countrycode', 'day'])['value_numeric'].mean().reset_index()
    
    # Merge the daily mean back to the original dataframe
    dashboard_df = dashboard_df.merge(daily_mean, on=['pollutant', 'network_countrycode', 'day'], suffixes=('', '_mean'))    

    dashboard_df['country'] = dashboard_df['network_countrycode'].map(country)
    dashboard_df = dashboard_df[['pollutant', 'country', 'month_day', 'value_numeric_mean','value_datetime_begin']].copy()

    dashboard_df = dashboard_df.drop_duplicates().reset_index(drop=True)
    dashboard_df = dashboard_df.sort_values('month_day')
    dashboard_df.to_csv('data_prova/se4g_dashboard_dataset_prova.csv', index=False)
    
else: 
        print("Dataset ",full_path," does not exist")
        print(os.getcwd())

In [10]:
file_path = 'data_prova/31-05-2023_09_15_34/ES_PM10.csv'
file_path = 'data_prova/31-05-2023_10_53_20/ES_PM10.csv'
print(file_path)

with open(file_path, 'r') as file:
    print(file)
    first_line = file.readline().strip()

if not first_line.startswith('<!DOCTYPE html'): 
    #print(fileName,'exist')

    df_temp = pd.read_csv(file_path)
    print(df_temp.head())

data_prova/31-05-2023_10_53_20/ES_PM10.csv
<_io.TextIOWrapper name='data_prova/31-05-2023_10_53_20/ES_PM10.csv' mode='r' encoding='utf-8'>
  network_countrycode network_localid    network_name network_namespace  \
0                  ES      NET_ES206A  CCAA Cantabria       ES.BDCA.AQD   
1                  ES      NET_ES206A  CCAA Cantabria       ES.BDCA.AQD   
2                  ES      NET_ES206A  CCAA Cantabria       ES.BDCA.AQD   
3                  ES      NET_ES206A  CCAA Cantabria       ES.BDCA.AQD   
4                  ES      NET_ES206A  CCAA Cantabria       ES.BDCA.AQD   

                                    network_timezone pollutant  \
0  http://dd.eionet.europa.eu/vocabulary/aq/timez...      PM10   
1  http://dd.eionet.europa.eu/vocabulary/aq/timez...      PM10   
2  http://dd.eionet.europa.eu/vocabulary/aq/timez...      PM10   
3  http://dd.eionet.europa.eu/vocabulary/aq/timez...      PM10   
4  http://dd.eionet.europa.eu/vocabulary/aq/timez...      PM10   

  samplingpoi

In [7]:
import codecs

In [11]:
with codecs.open('data_prova/31-05-2023_10_53_20/ES_PM10.csv', 'r', encoding='utf-8-sig') as file:
    content = file.read()

# Modify the content as needed

with codecs.open('data_prova/31-05-2023_10_53_20/ES_PM10.csv', 'w', encoding='utf-8') as file:
    file.write(content)

In [ ]:
import requests
def download_request(COUNTRIES= 'ES',
                        POLLUTANTS= 'PM10',
                        folder_out = 'data_prova'):
    print ('-----------------------------------------------------------------------')
    # Set download url
    # https://discomap.eea.europa.eu/map/fme/AirQualityUTDExport.htm
    ServiceUrl = "http://discomap.eea.europa.eu/map/fme/latest"


    dir = datetime.now().strftime("%d-%m-%Y_%H_%M_%S")

    if not os.path.exists(os.path.join(folder_out, dir)):
        if not os.path.exists(folder_out):
            os.mkdir(folder_out)
        os.mkdir(os.path.join(folder_out, dir))
        print(dir,'directory created')
        
    for country in COUNTRIES:
        for pollutant in POLLUTANTS:
            fileName = "%s_%s.csv" % (country, pollutant)
            downloadFile = '%s/%s_%s.csv' % (ServiceUrl, country, pollutant)
            #Download and save to local path
            print('Downloading: %s' % downloadFile )

            file = requests.get(downloadFile).content
            full_file = os.path.join(folder_out, dir, fileName)
			


            '''with codecs.open(full_file, 'r', encoding='utf-8-sig') as file:
                content = file.read()'''

            # Modify the content as needed
            with codecs.open(full_file, 'w', encoding='utf-8') as output:
                output.write(file)



            '''output = open(full_file, 'wb')
            output.write(file)
            output.close()'''
            print ('Saved locally as: %s ' % full_file)
            print ('-----')
    print ('Download finished')
    return dir

dirr = download_request()

In [14]:
def download_request(COUNTRIES=['ES'], POLLUTANTS=['PM10'], folder_out='data_prova'):
    print('-----------------------------------------------------------------------')
    # Set download url
    # https://discomap.eea.europa.eu/map/fme/AirQualityUTDExport.htm
    ServiceUrl = "http://discomap.eea.europa.eu/map/fme/latest"

    dir = datetime.now().strftime("%d-%m-%Y_%H_%M_%S")

    if not os.path.exists(os.path.join(folder_out, dir)):
        if not os.path.exists(folder_out):
            os.mkdir(folder_out)
        os.mkdir(os.path.join(folder_out, dir))
        print(dir, 'directory created')

    for country in COUNTRIES:
        for pollutant in POLLUTANTS:
            fileName = "%s_%s.csv" % (country, pollutant)
            downloadFile = '%s/%s_%s.csv' % (ServiceUrl, country, pollutant)
            # Download and save to local path
            print('Downloading: %s' % downloadFile)

            file_content = requests.get(downloadFile).content
            file_content_str = file_content.decode('utf-8-sig')

            full_file = os.path.join(folder_out, dir, fileName)

            with open(full_file, 'w', encoding='utf-8') as output:
                output.write(file_content_str)

            print('Saved locally as: %s ' % full_file)
            print('-----')
    print('Download finished')
    return dir

dirr = download_request()

-----------------------------------------------------------------------
31-05-2023_11_47_40 directory created
Downloading: http://discomap.eea.europa.eu/map/fme/latest/ES_PM10.csv
Saved locally as: data_prova\31-05-2023_11_47_40\ES_PM10.csv 
-----
Download finished


In [15]:
def download_request(COUNTRIES=['ES'], POLLUTANTS=['PM10'], folder_out='data_prova'):
    print('-----------------------------------------------------------------------')
    # Set download url
    # https://discomap.eea.europa.eu/map/fme/AirQualityUTDExport.htm
    ServiceUrl = "http://discomap.eea.europa.eu/map/fme/latest"

    dir = datetime.now().strftime("%d-%m-%Y_%H_%M_%S")

    if not os.path.exists(os.path.join(folder_out, dir)):
        if not os.path.exists(folder_out):
            os.mkdir(folder_out)
        os.mkdir(os.path.join(folder_out, dir))
        print(dir, 'directory created')

    for country in COUNTRIES:
        for pollutant in POLLUTANTS:
            fileName = "%s_%s.csv" % (country, pollutant)
            downloadFile = '%s/%s_%s.csv' % (ServiceUrl, country, pollutant)
            # Download and save to local path
            print('Downloading: %s' % downloadFile)

            file_content = requests.get(downloadFile).content
            file_content_str = file_content.decode('utf-8-sig')

            full_file = os.path.join(folder_out, dir, fileName)

            with codecs.open(full_file, 'w', encoding='utf-8') as output:
                output.write(file_content_str)

            print('Saved locally as: %s ' % full_file)
            print('-----')
    print('Download finished')
    return dir

dirr = download_request()

-----------------------------------------------------------------------
31-05-2023_11_47_59 directory created
Downloading: http://discomap.eea.europa.eu/map/fme/latest/ES_PM10.csv
Saved locally as: data_prova\31-05-2023_11_47_59\ES_PM10.csv 
-----
Download finished
